<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">

# Project 2

### Iowa Liquor Sales 

---

In this project, you will be using the pandas data transformation functionality (melt, pivot, groupby, apply) that was not a factor in the first half of Project 2. 

---

#### Context of the data

The state of Iowa provides many data sets on their website, including [this dataset](https://www.dropbox.com/sh/pf5n5sgfgiri3i8/AACkaMeL_i_WgZ00rpxOOcysa?dl=0) which contains transactions for all stores that have a class E liquor license. You can choose one of the following two scenarios.

The data can also be found [directly on their website](https://data.iowa.gov/Economy/Iowa-Liquor-Sales/m3tr-qhgy), which allows you to explore it graphically and download it (though it doesn't work very well).

NOTE: Some of you may have computer issues with the full dataset. In this case, feel free to use [this 10% dataset version of Iowa liquor sales](https://drive.google.com/file/d/0Bx2SHQGVqWaseDB4QU9ZSVFDY2M/view?usp=sharing). You may want to use it anyway to test and prototype your code since it will be faster, before running it on the full dataset.

---

#### Package imports

In [77]:
import numpy as np
import scipy.stats as stats
import pandas as pd

# this line tells jupyter notebook to put the plots in the notebook rather than saving them to file.
%matplotlib inline

# this line makes plots prettier on mac retina screens. If you don't have one it shouldn't do anything.
%config InlineBackend.figure_format = 'retina'

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 1. Load the data from file

---

In [95]:
# check the iowa file location for your computer 
iowa_file = '../../datasets/iowa_liquor/Iowa_Liquor_sales_sample_10pct.csv'

iowa = pd.read_csv(iowa_file)
print "Dataframe is of size: " + str(iowa.values.nbytes / 10**6) + "MB"

Dataframe is of size: 39MB


<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 2. Do an initial overview of the data

---

At the very least describe the columns/variables and the datatypes. 

In [96]:
print iowa.head()
iowa.describe().T

         Date  Store Number         City Zip Code  County Number      County  \
0  11/04/2015          3717       SUMNER    50674            9.0      Bremer   
1  03/02/2016          2614    DAVENPORT    52807           82.0       Scott   
2  02/11/2016          2106  CEDAR FALLS    50613            7.0  Black Hawk   
3  02/03/2016          2501         AMES    50010           85.0       Story   
4  08/18/2015          3654      BELMOND    50421           99.0      Wright   

    Category              Category Name  Vendor Number  Item Number  \
0  1051100.0           APRICOT BRANDIES             55        54436   
1  1011100.0           BLENDED WHISKIES            395        27605   
2  1011200.0  STRAIGHT BOURBON WHISKIES             65        19067   
3  1071100.0         AMERICAN COCKTAILS            395        59154   
4  1031080.0             VODKA 80 PROOF            297        35918   

            Item Description  Bottle Volume (ml) State Bottle Cost  \
0  Mr. Boston Apricot 

,count,mean,std,min,25%,50%,75%,max
Store Number,270955.0,3.590264e+03,947.662050,2106.00,2604.0,3722.00,4378.00,9023.00
County Number,269878.0,5.723164e+01,27.341205,1.00,NaN,NaN,NaN,99.00
Category,270887.0,1.043888e+06,50182.111075,1011100.00,NaN,NaN,NaN,1701100.00
Vendor Number,270955.0,2.564344e+02,141.014890,10.00,115.0,260.00,380.00,978.00
Item Number,270955.0,4.597496e+04,52757.043086,168.00,26827.0,38176.00,64573.00,995507.00
Bottle Volume (ml),270955.0,9.248303e+02,493.088489,50.00,750.0,750.00,1000.00,6000.00
Bottles Sold,270955.0,9.871285e+00,24.040912,1.00,2.0,6.00,12.00,2508.00
Volume Sold (Liters),270955.0,8.981351e+00,28.913690,0.10,1.5,5.25,10.50,2508.00
Volume Sold (Gallons),270955.0,2.372830e+00,7.638182,0.03,0.4,1.39,2.77,662.54


<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 3. Clean the dataset

---

### 3.1 Identify columns that you will need to convert and clean. Where and how is the data corrupted?

Don't worry about converting the date column to a pandas/numpy "datetime" datatype, unless you want to (not required for these problems and is a challenging thing to work with in its own right.)

In [97]:
iowa.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270955 entries, 0 to 270954
Data columns (total 18 columns):
Date                     270955 non-null object
Store Number             270955 non-null int64
City                     270955 non-null object
Zip Code                 270955 non-null object
County Number            269878 non-null float64
County                   269878 non-null object
Category                 270887 non-null float64
Category Name            270323 non-null object
Vendor Number            270955 non-null int64
Item Number              270955 non-null int64
Item Description         270955 non-null object
Bottle Volume (ml)       270955 non-null int64
State Bottle Cost        270955 non-null object
State Bottle Retail      270955 non-null object
Bottles Sold             270955 non-null int64
Sale (Dollars)           270955 non-null object
Volume Sold (Liters)     270955 non-null float64
Volume Sold (Gallons)    270955 non-null float64
dtypes: float64(4), int64(

### 3.2 Clean the columns

In [98]:
#method1 - demora muito o resultado, consome muita memoria 

#for i in range(len(sales)):
#    new_l.append(sales[i][1:(len(sales[i]))])
#print new_l

#method2 - demora para cacete tb!!!!!
sales = iowa['Sale (Dollars)']
new_l = [sales[i].replace('$','') for i in range(sales.shape[0])]
sales = new_l
sales = map(float, sales)
iowa['Sale (Dollars)'] = sales
iowa.rename(columns={'Sale (Dollars)': 'Sales'}, inplace=True)

sales = iowa['State Bottle Cost'] 
new_l = [sales[i].replace('$','') for i in range(sales.shape[0])]
sales = new_l
sales = map(float, sales)
iowa['State Bottle Cost'] = sales

sales = iowa['State Bottle Retail'] 
new_l = [sales[i].replace('$','') for i in range(sales.shape[0])]
sales = new_l
sales = map(float, sales)
iowa['State Bottle Retail'] = sales

#data_dropped=data.dropna(subset=['pick'])

iowa = iowa.dropna(subset=['Category'])
iowa = iowa.dropna(subset=['County Number'])
iowa.Date = iowa.Date.apply(pd.to_datetime)
iowa['strmy']=iowa['Date'].dt.strftime('%Y%b')
iowa.head(136)

,Date,Store Number,City,Zip Code,County Number,County,Category,Category Name,Vendor Number,Item Number,Item Description,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sales,Volume Sold (Liters),Volume Sold (Gallons),strmy
0,2015-11-04,3717,SUMNER,50674,9.0,Bremer,1051100.0,APRICOT BRANDIES,55,54436,Mr. Boston Apricot Brandy,750,4.50,6.75,12,81.00,9.00,2.38,2015Nov
1,2016-03-02,2614,DAVENPORT,52807,82.0,Scott,1011100.0,BLENDED WHISKIES,395,27605,Tin Cup,750,13.75,20.63,2,41.26,1.50,0.40,2016Mar
2,2016-02-11,2106,CEDAR FALLS,50613,7.0,Black Hawk,1011200.0,STRAIGHT BOURBON WHISKIES,65,19067,Jim Beam,1000,12.59,18.89,24,453.36,24.00,6.34,2016Feb
3,2016-02-03,2501,AMES,50010,85.0,Story,1071100.0,AMERICAN COCKTAILS,395,59154,1800 Ultimate Margarita,1750,9.50,14.25,6,85.50,10.50,2.77,2016Feb
4,2015-08-18,3654,BELMOND,50421,99.0,Wright,1031080.0,VODKA 80 PROOF,297,35918,Five O'clock Vodka,1750,7.20,10.80,12,129.60,21.00,5.55,2015Aug
5,2015-04-20,2569,CEDAR RAPIDS,52402,57.0,Linn,1041100.0,AMERICAN DRY GINS,205,31473,New Amsterdam Gin,1750,13.32,19.98,6,119.88,10.50,2.77,2015Apr
6,2015-08-05,2596,OTTUMWA,52501,90.0,Wapello,1051010.0,AMERICAN GRAPE BRANDIES,85,52806,Korbel Brandy,750,6.66,9.99,3,29.97,2.25,0.59,2015Aug
7,2015-06-25,3456,CLEAR LAKE,50428,17.0,Cerro Gordo,1012100.0,CANADIAN WHISKIES,65,10628,Canadian Club Whisky,1750,15.75,23.63,2,47.26,3.50,0.92,2015Jun
8,2016-01-04,4757,BONDURANT,50035,77.0,Polk,1032080.0,IMPORTED VODKA,370,34006,Absolut Swedish Vodka 80 Prf,750,11.49,17.24,4,68.96,3.00,0.79,2016Jan
9,2015-11-10,4346,SHELLSBURG,52332,6.0,Benton,1081315.0,CINNAMON SCHNAPPS,65,82610,Dekuyper Hot Damn!,1000,7.62,11.43,2,22.86,2.00,0.53,2015Nov


In [100]:
iowa.reset_index(inplace=True)
iowa.head(136)

,level_0,index,Date,Store Number,City,Zip Code,County Number,County,Category,Category Name,...,Item Number,Item Description,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sales,Volume Sold (Liters),Volume Sold (Gallons),strmy
0,0,0,2015-11-04,3717,SUMNER,50674,9.0,Bremer,1051100.0,APRICOT BRANDIES,...,54436,Mr. Boston Apricot Brandy,750,4.50,6.75,12,81.00,9.00,2.38,2015Nov
1,1,1,2016-03-02,2614,DAVENPORT,52807,82.0,Scott,1011100.0,BLENDED WHISKIES,...,27605,Tin Cup,750,13.75,20.63,2,41.26,1.50,0.40,2016Mar
2,2,2,2016-02-11,2106,CEDAR FALLS,50613,7.0,Black Hawk,1011200.0,STRAIGHT BOURBON WHISKIES,...,19067,Jim Beam,1000,12.59,18.89,24,453.36,24.00,6.34,2016Feb
3,3,3,2016-02-03,2501,AMES,50010,85.0,Story,1071100.0,AMERICAN COCKTAILS,...,59154,1800 Ultimate Margarita,1750,9.50,14.25,6,85.50,10.50,2.77,2016Feb
4,4,4,2015-08-18,3654,BELMOND,50421,99.0,Wright,1031080.0,VODKA 80 PROOF,...,35918,Five O'clock Vodka,1750,7.20,10.80,12,129.60,21.00,5.55,2015Aug
5,5,5,2015-04-20,2569,CEDAR RAPIDS,52402,57.0,Linn,1041100.0,AMERICAN DRY GINS,...,31473,New Amsterdam Gin,1750,13.32,19.98,6,119.88,10.50,2.77,2015Apr
6,6,6,2015-08-05,2596,OTTUMWA,52501,90.0,Wapello,1051010.0,AMERICAN GRAPE BRANDIES,...,52806,Korbel Brandy,750,6.66,9.99,3,29.97,2.25,0.59,2015Aug
7,7,7,2015-06-25,3456,CLEAR LAKE,50428,17.0,Cerro Gordo,1012100.0,CANADIAN WHISKIES,...,10628,Canadian Club Whisky,1750,15.75,23.63,2,47.26,3.50,0.92,2015Jun
8,8,8,2016-01-04,4757,BONDURANT,50035,77.0,Polk,1032080.0,IMPORTED VODKA,...,34006,Absolut Swedish Vodka 80 Prf,750,11.49,17.24,4,68.96,3.00,0.79,2016Jan
9,9,9,2015-11-10,4346,SHELLSBURG,52332,6.0,Benton,1081315.0,CINNAMON SCHNAPPS,...,82610,Dekuyper Hot Damn!,1000,7.62,11.43,2,22.86,2.00,0.53,2015Nov


<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 3.3 Perform more extensive cleaning of the dataset

Cleaning of data can mean a lot more than just fixing strings and numbers in columns. There are often logical errors with data, useless or nonsensical categories, redundancy of information, outliers, and many more problems.

This dataset has problems beyond just fixing the types of columns. Though resolving them may not be required for EDA and analysis, if you want experience with "deeper" cleaning of data this is a great dataset to start practicing with.

Keep in mind that some types of "data cleaning" is subjective: it's not always a cut-and-dry conversion of type or removal of null values. Subjectivity when dealing with data is just a fact of life for a data scientist. This isn't a kind of programming where things are just right or wrong.

In [101]:

iowa.shape

(269810, 21)

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 4. Filter/adjust the store data

---

Some stores may have opened or closed in 2015. These stores will have incorrect yearly summary statistics since they were not open the full year. We need to filter them out or find another way to deal with the inconsistent numbers of months across stores.

It is up to you how you want to deal with this problem.

1. Investigate problematic stores.
2. Decide on an approach to deal with stores not open for the full 2015 year.
3. Implement your plan.
4. Briefly report on what you did and why.


In [123]:
iowa_stores_2015 = iowa[(iowa['Date'].dt.year==2015)]
iowa_stores_2016 = iowa[(iowa['Date'].dt.year==2016)]
st_2015 = iowa_stores_2015['Store Number']
st_2016 = iowa_stores_2016['Store Number']

store_open=[]
for i in range(st_2015.shape[0]):
    if st_2015.values[i] in st_2016.values and st_2015.values[i] not in store_open:
        store_open.append(st_2015.values[i])

print store_open        
l_store_closed=[]
for i in range(iowa.shape[0]):
    if iowa['Store Number'][i] in store_open:
        l_store_closed.append(1)
    else:
        l_store_closed.append(0)

iowa['St Operational'] = l_store_closed

closed= iowa[(iowa['St Operational']==0)]
operational= iowa[(iowa['St Operational']==1)]

[3717, 3654, 2569, 2596, 3456, 4346, 2648, 2600, 3858, 4604, 2614, 2590, 3942, 3981, 4376, 3390, 3565, 3963, 4375, 2545, 4438, 2555, 2549, 2644, 4195, 2626, 2532, 4520, 3808, 2514, 4674, 2502, 4298, 3713, 4236, 3612, 4158, 2602, 2623, 2552, 3525, 2578, 4019, 4919, 4068, 4892, 3443, 2191, 2568, 4648, 3917, 4495, 4979, 3879, 4167, 2571, 3461, 4097, 4935, 5163, 5142, 4762, 2521, 2662, 5110, 2558, 4152, 4509, 4136, 2512, 3870, 2501, 2513, 2478, 3162, 3987, 5128, 2573, 4631, 2665, 3776, 4829, 4847, 3993, 4419, 2616, 4110, 4898, 3756, 4470, 2506, 4959, 2851, 5149, 3890, 4617, 4180, 2515, 5124, 4156, 2629, 2587, 2633, 4129, 4601, 3868, 4141, 3514, 3045, 4807, 3943, 4678, 2528, 2353, 4472, 3628, 4512, 2508, 2605, 4021, 2556, 2544, 2535, 4794, 4344, 5070, 2601, 2560, 2594, 2607, 2567, 2564, 2670, 5020, 4558, 2200, 4731, 4223, 3573, 3723, 2548, 4366, 3721, 2625, 4312, 2612, 3644, 4067, 3857, 5014, 4626, 4445, 2459, 4408, 5132, 4962, 3820, 2565, 3912, 4773, 4345, 2642, 4819, 3886, 2582, 5105, 266

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 5. Proposals for new tax rates (Scenario 1)

---

You are a data scientist in residence at the Iowa State tax board. The Iowa State legislature is considering changes in the liquor tax rates and has assigned you to the project.

### 5.1 Calculate yearly liquor sales for each store in 2015.

In [126]:
operational[(operational['Date'].dt.year == 2015)].groupby(['Store Number'])['Sales'].sum().sort_values(ascending=False)

Store Number
2633    1000783.30
4829     878812.70
3385     441983.55
2512     419746.83
3420     352020.66
3952     333311.98
3814     247417.42
3354     240365.94
3773     232460.41
2670     227167.07
2593     191333.88
3447     187650.04
2629     184574.96
5102     184330.16
2625     182118.58
3524     180805.62
2663     175203.29
3820     172792.22
4167     167173.55
2648     158428.10
2561     157134.87
2616     154160.15
4677     151432.73
2502     150094.01
2500     149121.63
2106     146326.22
4312     145363.04
2501     144833.66
2619     143958.96
2506     143431.66
           ...    
4338        890.10
4992        851.34
5047        795.06
5204        790.44
5160        786.72
5087        766.41
4996        756.42
4858        711.53
5196        682.85
5118        614.57
5189        604.20
5006        576.18
3677        573.51
5177        573.21
4985        531.96
4929        528.24
5152        520.74
5212        504.54
5206        500.90
4737        493.56
5020        472.08

### 5.2 Calculate the profit each store is making in 2015.


In [128]:
operational['margin_store'] = operational['State Bottle Retail'].sub(operational['State Bottle Cost'], axis=0)
operational['profit_store']=operational['margin_store'] * operational['Bottles Sold']

operational[(operational['Date'].dt.year == 2015)].groupby(['Store Number'])['profit_store'].sum().sort_values(ascending=False)

/Users/ritapaixao/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/ritapaixao/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


Store Number
2633    333816.91
4829    293196.99
3385    147884.62
2512    140143.46
3420    117593.34
3952    111230.21
3814     83004.48
3354     80244.69
3773     77525.08
2670     75853.72
2593     64197.45
3447     62683.22
2629     61636.70
5102     61504.64
2625     60794.18
3524     60528.54
2663     58512.28
3820     57650.18
4167     55756.18
2648     52870.35
2561     52844.61
2616     51472.43
4677     50517.91
2502     50345.31
2500     49801.48
2106     48838.08
4312     48493.09
2501     48430.73
2619     48105.29
2506     47925.66
          ...    
4338       296.74
4992       283.98
5047       265.08
5160       263.88
5204       263.78
5087       255.58
4996       252.28
4858       237.99
5196       227.70
5118       204.94
5189       201.42
5006       192.14
3677       191.64
5177       191.16
4985       177.32
4929       176.16
5152       173.68
5212       168.28
5206       167.02
4737       164.64
5020       157.56
4961       150.09
5178       112.19
5191       101.

### 5.3 Investigate which Iowa counties are making the most profit on liquor per gallon in 2015.

In [129]:
'Volume Sold (Gallons)'
operational[('profit_per_g')] =operational['profit_store'] / operational['Volume Sold (Gallons)']

operational[(operational['Date'].dt.year == 2015)].groupby(['County'])['profit_per_g'].sum().sort_values(ascending=False)

/Users/ritapaixao/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


County
Polk             961910.925707
Linn             435712.907163
Scott            317336.328963
Black Hawk       280733.063321
Johnson          250766.999791
Story            164517.030899
Pottawattamie    161857.043688
Woodbury         161820.734202
Dubuque          139019.053026
Cerro Gordo      112952.262011
Des Moines        77665.972248
Muscatine         71798.246307
Clinton           64193.858399
Dickinson         63699.908159
Lee               63662.814304
Wapello           62555.723385
Webster           54843.847901
Marshall          51601.558781
Buena Vista       51044.581076
Dallas            49894.659563
Jasper            47688.148479
Marion            46750.847381
Bremer            38040.324540
Poweshiek         37369.359859
Warren            37220.494454
Boone             34655.949919
Carroll           32524.406430
Clay              31951.576831
Washington        29555.273793
O'Brien           28897.812539
                     ...      
Cherokee          11257.074721
H

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 5.4 Plot a color map of average profit per county in 2015.

[Check out this site](http://flowingdata.com/2009/11/12/how-to-make-a-us-county-thematic-map-using-free-tools/) for  information on how to create map visualizations in python. There are other resources online for map plotting in python as well.

Another interesting map plot might show the difference in profit between time periods. Change in profit from November to December, for example.

### 5.5  Does profit scale proportionally to store size?

Size of a store can be defined as the total gallons sold in 2015. Is the profit to store size ratio increasing as store size increases? Decreasing? Consistent?

### 5.6 Create a broader category for liquor type.

Liquor types are pretty granular in this dataset. Create a column that categorizes these types into a smaller amount of categories. The categories you decide on are up to you.

### 5.7 Do relationships exist between the broader liquor type and profit?

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 5.8 Are there any noticeable relationships between liquor brand and sales? Profit?

### 5.9 The tax board wants to design a tax or taxes that affect larger stores more than smaller "mom and pop" stores.

Based on your investigations into the data, come up with a way you could design a tax that achieves this goal **without explicitly taxing stores based on size or county critera.** The liqour board does not want to obviously punish larger stores or speific counties for fear of backlash, but is willing to tax hard alcohol more than beer, for example.

Feel free to do more EDA if it helps.

Your report should describe whether such a tax is possible or not, and the specifics of what the tax will target/do.

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 6. Market research for new store locations (Scenario 2)

A liquor store owner in Iowa is looking to open a store in a new location and has hired you to investigate the market data for potential new locations. The business owner is interested in what you can discover from the data that will help him evaluate a good location for a new storefront.

---

### 6.1 Create columns for Q1 2015 sales and Q1 2016 sales.

The owner is not just interested in what locations are currently selling well, but also which areas are showing the most promising sales growth from 2015 to 2016. 

For yearly changes we will focus on the changes from Q1 2015 to Q1 2016 specifically (avoiding effects of season changes).

Note: Q1 is months: January, February, and March.


In [137]:
operational.reset_index(inplace=True)
l_q1_2015 = []
for i in range(operational.shape[0]):
    if operational['strmy'][i]=="2015Jan" or operational['strmy'][i]=="2015Feb" or operational['strmy'][i]=="2015Mar":
        l_q1_2015.append(1)
    else:
        l_q1_2015.append(0)

l_q1_2016 = []
for i in range(operational.shape[0]):
    if operational['strmy'][i]=="2016Jan" or operational['strmy'][i]=="2016Feb" or operational['strmy'][i]=="2016Mar":
        l_q1_2016.append(1)
    else:
        l_q1_2016.append(0)

operational['Q1 2015'] = l_q1_2015
operational['Q1 2016'] = l_q1_2016
operational.head()



/Users/ritapaixao/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/ritapaixao/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,index,Date,Store Number,City,Zip Code,County Number,County,Category,Category Name,Vendor Number,...,Sales,Volume Sold (Liters),Volume Sold (Gallons),strmy,St Operational,margin_store,profit_store,profit_per_g,Q1 2015,Q1 2016
0,0,2015-11-04,3717,SUMNER,50674,9.0,Bremer,1051100.0,APRICOT BRANDIES,55,...,81.00,9.0,2.38,2015Nov,1,2.25,27.00,11.344538,0,0
1,1,2016-03-02,2614,DAVENPORT,52807,82.0,Scott,1011100.0,BLENDED WHISKIES,395,...,41.26,1.5,0.40,2016Mar,1,6.88,13.76,34.400000,0,1
2,2,2016-02-11,2106,CEDAR FALLS,50613,7.0,Black Hawk,1011200.0,STRAIGHT BOURBON WHISKIES,65,...,453.36,24.0,6.34,2016Feb,1,6.30,151.20,23.848580,0,1
3,3,2016-02-03,2501,AMES,50010,85.0,Story,1071100.0,AMERICAN COCKTAILS,395,...,85.50,10.5,2.77,2016Feb,1,4.75,28.50,10.288809,0,1
4,4,2015-08-18,3654,BELMOND,50421,99.0,Wright,1031080.0,VODKA 80 PROOF,297,...,129.60,21.0,5.55,2015Aug,1,3.60,43.20,7.783784,0,0


### 6.2 Decide on a metric for performance that combines Q1 2016 sales as well as growth in sales from Q1 2015.

Again, the owner wants to know that his store will be in a good position to sell a large amount of liquor now, but also have his store be more likely to grow and sell even more going forward. 

In particular, he is worried about opening a store in an area that is showing a decline in liquor sales between Q1 2015 and Q1 2016.

In [156]:
y2015= operational[(operational['Q1 2015']==1)].sort_values("profit_per_g",ascending=False).head(15)
y2016= operational[(operational['Q1 2016']==1)].sort_values("profit_per_g",ascending=False).head(15)

print y2015
print y2016

#pd.pivot_table(y2015,index=["County"],values=['Sales']])


#data = pd.concat([y2015, y2016])


         index       Date  Store Number          City Zip Code  County Number  \
191781  195509 2015-02-26          2663     URBANDALE    50322           77.0   
213203  217337 2015-02-24          4042      GLENWOOD    51534           65.0   
175176  178585 2015-03-30          2633    DES MOINES    50320           77.0   
178079  181539 2015-01-12          3773  CEDAR RAPIDS    52401           57.0   
256459  261484 2015-03-11          2576    STORM LAKE    50588           11.0   
208639  212674 2015-03-26          2595       DENISON    51442           24.0   
62659    63913 2015-01-15          4829    DES MOINES    50314           77.0   
122532  124933 2015-01-21          4879    SIOUX CITY    51106           97.0   
22609    23042 2015-03-04          2625     DAVENPORT    52804           82.0   
74150    75658 2015-03-11          2623    SIOUX CITY    51104           97.0   
66890    68224 2015-01-07          2465       DUBUQUE    52003           31.0   
27069    27589 2015-01-28   

### 6.3 Recommend a location based on your metric.

The owner asks you to make a recommendation for a new store location. Explain why you recommended this area.

### 6.4 Recommend a location based also on a restricted inventory type.

Your employer has now decided he wants his store to focus on one type of liquor only. He askes you:

1. What would be the best type of liquor to sell to maximize profit?
2. Should the recommended location change if he is only selling one type of liquor?

<img src="http://imgur.com/GCAf1UX.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 7. Time-related effects

---

You could imagine that liquor sales might be affected by a variety of effects related to time. Do people buy more beer in the summer? Do liquor sales skyrocket in december? Do people buy less liquor on Tuesdays?

You have the date of sales in your dataset, which you can use to pull out time components.

1. Come up with 3 different hypotheses about how liquor sales may vary with time-related variables. 
2. Create a visualization exploring each hypothesis.
3. Write brief concluding remarks on what you observed.


<img src="http://imgur.com/GCAf1UX.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 8. Appropriate spending

---

The owner you worked with before realized that opening new stores cost money, something he had completely overlooked earlier. He brings you back in to help him figure out how much he should be willing to spend on opening his store.

### 8.1 Calculate costs and time to recoup.

The owner has decided he will buy 20,000 gallons of the liquor type you recommended (randomly sampled across the different brands/subtypes). He will also be opening his store in the area you recommended.

The cost of buying the property for his store will be $150,000.

1. How much should he expect to spend on the 20,000 gallons of liquor alone?
2. Based on the amount he's spent on the liquor and property, and on the profit per time period he is expected to have, how long will it take him to recoup the costs?

<img src="http://imgur.com/gGrau8s.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">


### 8.2 Calculate maximum loan interest rate.

The owner unfortunately has to take out a loan to to pay for the entirety of the property and liquor. 

**Conditions of the loan**:

Every quarter, (except for Q1 when he takes out the loan), the interest rate is applied to the remaining balance on the loan and added to his debt. 

If, for example, he made no payments whatsoever, not only would the amount of debt he owed keep increasing by quarter but the debt incurred from interest would keep increasing as well. (Since his debt keeps growing, the interest rate is applied each month to the bigger balance, incurring more debt each time.)

**Determining maximum acceptable interest:**

Given this, _the owner wants to be 99% confident that he will be able to pay off his loans._ If the interest rate is too high he will have to eventually declare bankruptcy.

Based on his expected profits per quarter (assume the expected profit stays the same for all future quarters), what is the maximum interest rate he should accept on the loan to be 99% confident he will be able to pay it off?